# Analyse des données de transport

## Objectifs

Ce notebool a pour objectif d'explorer et d'analyser un historique de tickets de transports:
* Analyser les prix et durée des trajets
* comparer les modes de transport selon la distance
* identifier des tendances et incohérences dans les données
* proposer des analyses et visualisations pertinentes

## Outils et bibliothèques utilisés
* Python
* pandas
* numpy
* matplotlib
* scikit-learn
* pyarrow
* pyjanitor
* ipykernel
* jupyter
* seaborn
* polars
* plotly
* folium
* missingno
* great_expectations

## 1. Import des librairies et configuration de l'apparence globale des graphiques

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style('whitegrid')

## 2. Chargement des données

In [61]:
ticket = pd.read_csv('data/ticket_data.csv')
cities = pd.read_csv('data/cities.csv')
stations = pd.read_csv('data/stations.csv')
providers = pd.read_csv('data/providers.csv')

###### Aperçu des données

In [62]:
from IPython.display import display, Markdown

display(Markdown("### Info sur les tickets"))
ticket.head()
ticket.info()

display(Markdown("### Info sur les villes"))
cities.head()
cities.info()

display(Markdown("### Info sur les stations"))
stations.head()
stations.info()

display(Markdown("### Info sur les fournisseurs"))
providers.head()
providers.info()

### Info sur les tickets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74168 entries, 0 to 74167
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               74168 non-null  int64  
 1   company          74168 non-null  int64  
 2   o_station        32727 non-null  float64
 3   d_station        32727 non-null  float64
 4   departure_ts     74168 non-null  object 
 5   arrival_ts       74168 non-null  object 
 6   price_in_cents   74168 non-null  int64  
 7   search_ts        74168 non-null  object 
 8   middle_stations  32727 non-null  object 
 9   other_companies  32727 non-null  object 
 10  o_city           74168 non-null  int64  
 11  d_city           74168 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 6.8+ MB


### Info sur les villes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8040 entries, 0 to 8039
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8040 non-null   int64  
 1   local_name   8040 non-null   object 
 2   unique_name  8039 non-null   object 
 3   latitude     8040 non-null   float64
 4   longitude    8040 non-null   float64
 5   population   369 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 377.0+ KB


### Info sur les stations

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11035 entries, 0 to 11034
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           11035 non-null  int64  
 1   unique_name  11035 non-null  object 
 2   latitude     11035 non-null  float64
 3   longitude    11035 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 345.0+ KB


### Info sur les fournisseurs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    227 non-null    int64 
 1   company_id            227 non-null    int64 
 2   provider_id           213 non-null    object
 3   name                  227 non-null    object
 4   fullname              227 non-null    object
 5   has_wifi              224 non-null    object
 6   has_plug              224 non-null    object
 7   has_adjustable_seats  224 non-null    object
 8   has_bicycle           224 non-null    object
 9   transport_type        227 non-null    object
dtypes: int64(2), object(8)
memory usage: 17.9+ KB


## 3. Compréhension métier des données

### Description des colonnes clés
* *price_in_cents* : prix du ticket en cents (€)
* *departure_ts / arrival_ts* : départ et arrivée : format data YYYY-mm-jj HH:mm:ss
* *o_city / d_city* : ville de départ / arrivée
* *o_station / d_station* : station de départ / arrivée
* *company* : provider (ex : TGV, TER, Blablacar)

## 4. Nettoyage de données

#### Valeurs manquantes

In [63]:
ticket.isna().sum()

id                     0
company                0
o_station          41441
d_station          41441
departure_ts           0
arrival_ts             0
price_in_cents         0
search_ts              0
middle_stations    41441
other_companies    41441
o_city                 0
d_city                 0
dtype: int64

#### Valeurs abérantes

Vérification de l'existance de données abérantes.
Comparaison de la forme des tableau avant et après filtrage des données abérantes.
Suppression des valeurs abérantes si nécessaire.

In [64]:
print('Avant nettoyage :', ticket.shape)
ticket_filter = ticket[(ticket['price_in_cents'] > 0)]
print('Après nettoyage :', ticket_filter.shape)

if ticket.shape[0] == ticket_filter.shape[0]:
    print("Aucun enregistrement supprimé.")
else:
    ticket = ticket[(ticket['price_in_cents'] > 0)]

Avant nettoyage : (74168, 12)
Après nettoyage : (74168, 12)
Aucun enregistrement supprimé.


#### Doublons

La suppression des doublons permettra d'éviter que les résultats seront biaisés.

In [65]:
ticket = ticket.drop_duplicates()

if ticket.drop_duplicates().shape[0] == ticket.shape[0]:
    print("Aucun doublon trouvé.")
else:
    print('Après suppression des doublons :', ticket.shape)

Aucun doublon trouvé.


## 5. Jointures des datasets

Jointure progressive (vile -> stations -> fournisseurs)

In [ ]:
print(Markdown("### Jointure des données des tickets avec les villes d'origine."))

cities_origin = cities[['id', 'local_name', 'latitude', 'longitude']].copy()
cities_origin = cities_origin.rename(columns={
    'id': 'city_id',
    'local_name': 'o_city_name',
    'latitude': 'o_city_latitude',
    'longitude': 'o_city_longitude'
})

"""Merge des données des tickets avec les villes d'origine."""
ticket = ticket.merge(cities_origin, left_on='o_city', right_on='city_id', how='left').drop(columns=['city_id'])


<IPython.core.display.Markdown object>


(74168, 18)

In [75]:
print(Markdown("### Jointure des données des tickets avec les villes de destinations"))

cities_destination = cities[['id', 'local_name', 'latitude', 'longitude']].copy()
cities_destination = cities_destination.rename(columns={
    'id': 'city_id',
    'local_name': 'd_city_name',
    'latitude': 'd_city_latitude',
    'longitude': 'd_city_longitude'
})

"""Merge des données des tickets avec les villes de destinations."""
ticket = ticket.merge(cities_destination, left_on='d_city', right_on='city_id', how='left').drop(columns=['city_id'])

<IPython.core.display.Markdown object>


In [76]:
print(Markdown("### Jointure des données avec les stations d'origine"))

stations_origin = stations[['id', 'unique_name', 'latitude', 'longitude']].copy()
stations_origin = stations_origin.rename(columns={
    'id': 'station_id',
    'unique_name': 'o_station_name',
    'latitude': 'o_station_latitude',
    'longitude': 'o_station_longitude'
})

"""Merge des données avec les stations d'origine."""
ticket = ticket.merge(stations_origin, left_on='o_station', right_on='station_id', how='left').drop(columns=['station_id'])

<IPython.core.display.Markdown object>


In [ ]:
print(Markdown("### Jointure des données avec les stations de destination"))

stations_destination = stations[['id', 'unique_name', 'latitude', 'longitude']].copy()
stations_destination = stations_destination.rename(columns={
    'id': 'station_id',
    'unique_name': 'd_station_name',
    'latitude': 'd_station_latitude',
    'longitude': 'd_station_longitude'
})

"""Merge des données avec les stations de destination."""
ticket = ticket.merge(stations_destination, left_on='d_station', right_on='station_id', how='left').drop(columns=['station_id'])